In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May 31 17:17:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/ML Final Project/Colab/"

/content/drive/.shortcut-targets-by-id/1UKmlnv56AciuvFxlMyNSXgaV23ugGklV/ML Final Project/Colab


YOLO setup

In [ ]:
# !nvidia-smi

In [ ]:
#prerequirements
# !sudo apt-get install tree -qq > /dev/null
# !pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install numpy==1.17
# !pip install PyYAML==5.3.1
# !pip install git+https://github.com/cocodataset/cocoapi.git

In [ ]:
## restart

In [ ]:
# !git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

YOLO dataset preparation

In [ ]:
# import data_preparation as dp
# from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np                               
import pandas as pd                        
import matplotlib.pyplot as plt                  
import cv2             
import tensorflow as tf                          
from PIL import Image                          
import os    
import json
%matplotlib inline

In [ ]:
import pickle
with open('datafile/data_preparation.pkl', 'rb') as pickle_in:
     names = pickle.load(pickle_in)
     coordinates = pickle.load(pickle_in)
     lights = pickle.load(pickle_in)
     

In [ ]:
categories = ['R', 'Y', 'G', 'NA']
categories.sort()
categories

['G', 'NA', 'R', 'Y']

Darknet label format: [label_index, xcenter, ycenter, w, h] (Relative coordinates)
For example:
xmin: top-left x coordinate,
ymin: top-left y coordinate,
w: bounding box width,
h: bounding box height,
w_img: image width,
h_img: image height

xcenter = (xmin + w/2) / w_img
ycenter = (ymin + h/2) / h_img
w = w / w_img
h = h / h_img

In [ ]:
def create_yolo_dataset(names, lights, coordinates, dataset_type):
  image_path = Path(f"yolo/images/{dataset_type}")
  image_path.mkdir(parents=True, exist_ok=True)

  label_path = Path(f"yolo/labels/{dataset_type}")
  label_path.mkdir(parents=True, exist_ok=True)

  # for img_id, row in enumerate(tqdm(names)):
  for i in range(len(names)):
    image_name = f"{names[i]}" 

    img = Image.open(f'datafile/image_200/{names[i]}')
    img=img.convert('RGB')

    img.save(str(image_path / image_name))

    label_name = f"{names[i].strip('.jpg')}.txt"

    with(label_path / label_name).open(mode="w") as label_file:
      for j in range(len(lights[i])):
        category_idx = categories.index(lights[i][j])

        x1, y1 = coordinates[i][j]['x1'], coordinates[i][j]['y1']
        x2, y2 = coordinates[i][j]['x2'], coordinates[i][j]['y2']

        
        xmin = x1
        ymin = y1
        w = x2-x1
        h = y2-y1
        w_img = 1280
        h_img = 720

        xcenter = (xmin + w/2) / w_img
        ycenter = (ymin + h/2) / h_img
        w = w / w_img
        h = h / h_img


        # assign different color to different classes of objects
        if category_idx == 1:
          edgecolor = 'B'
          # ax.annotate('NA', xy=(xmax-40,ymin+20))
        elif category_idx == 2:
          edgecolor = 'R'
          # ax.annotate('R')
        elif category_idx ==3:
          edgecolor = 'Y'
          # ax.annotate('Y', xy=(xmax-40,ymin+20))
        elif category_idx == 0:
          edgecolor = 'G'
          # ax.annotate('G', xy=(xmax-40,ymin+20))

        label_file.write(f"{category_idx} {xcenter} {ycenter} {w} {h}\n")


In [ ]:
# train_set, val_set = train_test_split(names, test_size=0.3)

In [ ]:
from random import seed, random
def random_split(names, lights, coordinates, train_size, seed_):
  seed(seed_)
  lst=[]
  for _ in range(len(names)):
    idx = random()
    lst.append(idx)

  train_name, train_label, train_box = names[:train_size], lights[:train_size], coordinates[:train_size]
  val_name, val_label, val_box = names[train_size:], lights[train_size:], coordinates[train_size:]

  return train_name, train_label, train_box, val_name, val_label, val_box

In [ ]:
train_name, train_label, train_box, val_name, val_label, val_box = random_split(names, lights, coordinates, 150, 1)

In [ ]:
print(len(train_name))
print(len(val_name))


150
50


In [ ]:
create_yolo_dataset(train_name, train_label, train_box, 'train') 
create_yolo_dataset(val_name, val_label, val_box, 'val')

In [ ]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.8.1+cu101 (Tesla P100-PCIE-16GB)


In [ ]:
# prepare the .yaml

In [ ]:
!python train.py --img 640 --batch 8 --epochs 300 --data ./data/lights.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt --name yolov5x_lights --cache

remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 25 (delta 15), reused 16 (delta 10), pack-reused 0
Unpacking objects: 100% (25/25), done.
From https://github.com/ultralytics/yolov5
   4b52e19..fdbe527  develop     -> origin/develop
 * [new branch]      multi-cache -> origin/multi-cache
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2021-5-29 torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=True, cfg='./models/yolov5x.yaml', data='./data/lights.yaml', device='', entity=None, epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov5x_lights', noautoanchor=False, nosave=False, notest=False, project

In [ ]:
# Start tensorboard
# # Launch after you have started training to all the graphs needed for inspection
# # logs save in the folder "runs"
# %load_ext tensorboard
# %tensorboard --logdir /content/yolov5/runs

In [ ]:
!python detect.py --weights runs/train/yolov5x_lights19/weights/best.pt --img 640 --conf 0.25 --source ../test_10.mp4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='../test_10.mp4', update=False, view_img=False, weights=['runs/train/yolov5x_lights19/weights/best.pt'])
YOLOv5 🚀 2021-5-29 torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 476 layers, 87218881 parameters, 0 gradients, 217.2 GFLOPS
video 1/1 (1/600) /content/drive/.shortcut-targets-by-id/1UKmlnv56AciuvFxlMyNSXgaV23ugGklV/ML Final Project/Colab/yolov5/../test_10.mp4: 384x640 4 Gs, Done. (0.036s)
video 1/1 (2/600) /content/drive/.shortcut-targets-by-id/1UKmlnv56AciuvFxlMyNSXgaV23ugGklV/ML Final Project/Colab/yolov5/../test_10.mp4: 384x640 5 Gs, Done. (0.034s)
video 1/1 (3/600) /content/drive/.shortcut-targets-by-id/1UKmlnv56AciuvFxlMyNSXgaV2

In [ ]:
!python train.py --img 640 --batch 8 --epochs 300 --data ./data/lights.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt --name yolov5x_lights --cache